In [9]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import os
import time
import requests
import warnings
warnings.filterwarnings('ignore')

In [11]:
!which chromedriver

/usr/local/bin/chromedriver


In [21]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [22]:
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)

In [23]:
#html
html = browser.html

# parsing w bs
soup = bs(html, 'html.parser')

# pulling news title and paragraph
news = soup.find("div", class_='list_text')
news_title = news.find("div", class_="content_title").text
news_para = news.find("div", class_ ="article_teaser_body").text

#print
print(news_title)
print(news_para)

NASA's Curiosity Keeps Rolling As Team Operates Rover From Home
The team has learned to meet new challenges as they work remotely on the Mars mission.


In [46]:
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)

image_full=browser.find_by_id('full_image')
image_full.click()
time.sleep(2)
browser.click_link_by_partial_text('more info')


html = browser.html
image_soup = bs(html, 'html.parser')


feat_img_url = image_soup.find('figure', class_='lede').a['href']
featured_image_url = f'https://www.jpl.nasa.gov{feat_img_url}'
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17932_hires.jpg


In [50]:
# Visit Twitter url for latest Mars Weather
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)
time.sleep(1)

html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Find all elements that contain tweets
latest_tweets = soup.find_all('div', class_='js-tweet-text-container')

# Retrieve all elements that contain news title in the specified range
# Look for entries that display weather related words to exclude non weather related tweets 
for tweet in latest_tweets: 
    mars_weather = tweet.find('p').text
    if 'Sol' and 'pressure' in mars_weather:
        print(mars_weather)
        break
    else: 
        pass

In [39]:
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)
html = browser.html

In [40]:
# Use Pandas to scrape the table containing facts about Mars
table = pd.read_html(facts_url)
mars_facts = table[1]

# Rename columns
mars_facts.columns = ['Description','Value']

# Reset Index to be description
mars_facts.set_index('Description', inplace=True)
mars_facts

,Value
Description,
Diameter:,"3,475 km"
Mass:,7.35 × 10^22 kg (0.01 Earths)
Orbits:,The Earth
Orbit Distance:,"384,400 km"
Orbit Period:,27.3 days
Surface Temperature:,-233 to 123 °C


In [41]:
# Use Pandas to convert the data to a HTML table string
mars_facts.to_html('table.html')

In [42]:
# Visit USGS webpage for Mars hemispehere images
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html = browser.html

In [43]:
# Parse HTML with Beautiful Soup
soup = bs(html, "html.parser")

# Create dictionary to store titles & links to images
hemisphere_image_urls = []

# Retrieve all elements that contain image information
results = soup.find("div", class_ = "result-list" )
hemispheres = results.find_all("div", class_="item")

# Iterate through each image
for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    hemisphere_image_urls.append({"title": title, "img_url": image_url})

# Print image title and url
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
